# Colab initializing

In [0]:
import torch
print('Version', torch.__version__)
print('CUDA enabled:', torch.cuda.is_available())
# !nvcc --version
# Running this should then print out:
# Version 1.3.0+cu100
# CUDA enabled: True

Version 1.3.1
CUDA enabled: True


In [0]:
from google.colab import drive
drive.mount('/gdrive')
!ls /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
'My Drive'


In [0]:
import os
BASE_PATH = '/gdrive/My Drive/colab_files/dl_project/'
if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)
DATA_PATH = '/content/'

os.chdir(BASE_PATH)
!cp pt_util.py /content

os.chdir('/content')

In [0]:
#Installing bert tokenizer (Used as vocabulary)
!pip install spacy ftfy==4.4.3
!python -m spacy download en
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for ftfy: filename=ftfy-4.4.3-cp36-none-any.whl size=41071 sha256=3ec6c2e8b33004ff85619a4159fc0f1d4d1f76cae56ef1ac994c7989d42976d8
  Stored in directory: /root/.cache/pip/wheels/37/54/00/d320239bfc8aad1455314f302dd82a75253fc585e17b81704e
Successfully built ftfy
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 645kB 8.7MB/s 


In [0]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
# from torchvision import datasets
# from torchvision import transforms
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import sys
import pickle
import re
import pt_util
from pandas import read_csv as rcsv
from pandas import read_pickle as rpkl
from pytorch_pretrained_bert import OpenAIGPTTokenizer

In [0]:
#Important variables to be used throughout the code
max_name_len=20
max_ingr_len=20   #maximum=18
max_ingr_inner_len=18   #maximum=17
max_steps_len=300

# Pre-processing data

In [0]:
def prepare_data(data_path):
    #Set the maximum lengths for names, ingredients and steps
    # max_name_len=20
    # max_ingr_len=20
    # max_ingr_inner_len=18
    # max_steps_len=300

    names=[]
    ingr=[]
    steps=[]
    cal=[]

    #Reading the PP_recipes.csv file
    pp_recipes = rcsv(data_path)

    temp_names=list(pp_recipes['name_tokens'])
    temp_ingr=list(pp_recipes['ingredient_tokens'])
    temp_cal=list(pp_recipes['calorie_level'])
    temp_steps=pp_recipes['steps_tokens'].tolist()
    temp_cal=pp_recipes['calorie_level'].tolist()

    #Getting integer lists from pandas.series.Series
    for i in temp_cal:
        cal.append(int(i))
    
    for i in temp_names:
        temp_i = i[1:-1].split(',')
        for j in range(len(temp_i)):
            temp_i[j]=int(temp_i[j])
        while (len(temp_i)<max_name_len):
            temp_i.append(0)
        names.append(temp_i)

    for i in temp_ingr:
        temp_i=[]
        for j in range(len(i[1:-1])):
            if i[j]=='[':
                start=j
            if i[j]==']':
                temp_i.append(i[start+1:j].split(','))
        for ii in range(len(temp_i)):
            for jj in range(len(temp_i[ii])):
                # print ('Value of jj', jj)
                # print ('Value of temp_i', temp_i[ii])
                temp_i[ii][jj]=int(temp_i[ii][jj])
            while (len(temp_i[ii])<max_ingr_inner_len):
                temp_i[ii].append(0)
        while (len(temp_i)<max_ingr_len):
            temp_i.append([0]*max_ingr_inner_len)
        ingr.append(temp_i)

    for i in temp_steps:
        temp_i=i[1:-1].split(',')
        for j in range(len(temp_i)):
            temp_i[j]=int(temp_i[j])
        if (len(temp_i)>max_steps_len):
            print ('Higher value',len(temp_i))
        while (len(temp_i)<max_steps_len):
            temp_i.append(0)
        steps.append(temp_i)
    
    batch_size=len(names)
    assert len(names)==len(ingr)
    assert len(names)==len(steps)
    assert len(ingr)==len(steps)
    assert len(names)==len(cal)

    # print ('Length', len(steps))
    # print ('Inside length', len(steps[100]))
    # print ('Inside length of ingr', ingr[0])
    # print ('Inside steps', steps[0][0:150])
    # 80% text
    train_size = int(0.8*batch_size)
    train_name = names[:train_size] # TODO Fill this in
    train_ingr = ingr[:train_size]
    train_steps = steps[:train_size]
    train_cal= cal[:train_size]

    test_name = names[train_size:] # TODO Fill this in
    test_ingr = ingr[train_size:]
    test_steps = steps[train_size:]
    test_cal= cal[train_size:]

    pickle.dump({'name_tokens': train_name, 'ingr_tokens': train_ingr, 'step_tokens':train_steps, 'cal_tokens':train_cal}, open(DATA_PATH + 'pp_recipes_chars_train.pkl', 'wb'))
    pickle.dump({'name_tokens': test_name, 'ingr_tokens': test_ingr, 'step_tokens':test_steps, 'cal_tokens':test_cal}, open(DATA_PATH + 'pp_recipes_chars_test.pkl', 'wb'))

prepare_data(BASE_PATH + 'pp_recipes.csv')

# Datasetloader

In [0]:
## Convert to batches given batch size and sequence length

class Datasetloader(torch.utils.data.Dataset):
    def __init__(self, data_file, sequence_length, batch_size, vocab_size):
        super(Datasetloader, self).__init__()

        self.sequence_length = sequence_length
        self.batch_size = batch_size
        self.data_file=data_file
        self.vocab_size=vocab_size

        with open(data_file, 'rb') as data_pkl:
            dataset = pickle.load(data_pkl)
        
        self.name_tokens=dataset['name_tokens']
        self.ingr_tokens=dataset['ingr_tokens']
        self.step_tokens=dataset['step_tokens']
        self.calorie_tokens=dataset['cal_tokens']

        N_original=len(self.name_tokens)

        #Making the datasets all a multiple of batch size
        a=int(N_original/self.batch_size)*self.batch_size

        if (N_original>a):
            self.name_tokens=self.name_tokens[:a]
            self.ingr_tokens=self.ingr_tokens[:a]
            self.step_tokens=self.step_tokens[:a]
            self.calorie_tokens=self.calorie_tokens[:a]

        assert len(self.ingr_tokens)==len(self.step_tokens)
        assert len(self.ingr_tokens)==len(self.name_tokens)
        assert len(self.ingr_tokens)==len(self.calorie_tokens)
        
        N_new = len(self.name_tokens)
        self.chunks = N_new//(self.batch_size*self.sequence_length)

        #Converting the dataset into batches
        name_temp = []
        ingr_temp = []
        step_temp = []
        calorie_temp = []
        data_divide = N_new//(self.batch_size) #Data starting points for every batch

        for ii in range(self.chunks):
            name_temp1 = []
            ingr_temp1 = []
            step_temp1 = []
            calorie_temp1 = []
            for jj in range(self.batch_size):
                name_temp2 = []
                ingr_temp2 = []
                step_temp2 = []
                calorie_temp2 = []
                for kk in range(self.sequence_length):
                    name_temp2.append(self.name_tokens[ii*self.sequence_length+jj*data_divide+kk])
                    ingr_temp2.append(self.ingr_tokens[ii*self.sequence_length+jj*data_divide+kk])
                    step_temp2.append(self.step_tokens[ii*self.sequence_length+jj*data_divide+kk])
                    calorie_temp2.append(self.calorie_tokens[ii*self.sequence_length+jj*data_divide+kk])
                name_temp1.append(name_temp2)
                ingr_temp1.append(ingr_temp2)
                step_temp1.append(step_temp2)
                calorie_temp1.append(calorie_temp2)
            name_temp.append(name_temp1)
            ingr_temp.append(ingr_temp1)
            step_temp.append(step_temp1)
            calorie_temp.append(calorie_temp1)

        # ingr_temp=np.asarray(ingr_temp, dtype=int)
        #Convert variables into torch tensors dtype torch.long
        self.name_tokens = torch.tensor(name_temp, dtype=torch.long).reshape(-1, max_name_len, self.sequence_length)
        self.ingr_tokens = torch.tensor(ingr_temp, dtype=torch.long).reshape(-1, max_ingr_len, max_ingr_inner_len, self.sequence_length)
        self.step_tokens = torch.tensor(step_temp, dtype=torch.long).reshape(-1, max_steps_len, self.sequence_length)
        self.calorie_tokens = torch.tensor(calorie_temp, dtype=torch.long).reshape(-1, self.sequence_length)

    def __len__(self):
        # TODO return the number of unique sequences you have, not the number of characters.
         return self.chunks*self.batch_size
        
    def __getitem__(self, idx):
        # Return the data and label for a character sequence as described above.
        # The data and labels should be torch long tensors.
        # You should return a single entry for the batch using the idx to decide which chunk you are 
        # in and how far down in the chunk you are.

        a = [0]*self.vocab_size   #Zeros of vocab_size
        for i in range(max_steps_len):
            for j in range(self.sequence_length):
                a[self.step_tokens[idx,i,j].item()]=1   #Sequence length is 1

        #Converting a to tensor
        a=torch.tensor(a, dtype=torch.float32)

        return (self.name_tokens[idx,:], self.calorie_tokens[idx,:], self.ingr_tokens[idx,:]), a

# a=Datasetloader(DATA_PATH+'pp_recipes_chars_train.pkl', 10, 100, 40483)   #sequence_length, batch_size

# b=a.__len__()
# print ('len out', b)

# c = a.__getitem__(7000)
# # print ('getitem out', c[20])
# print ('first element', c[0][0].shape)
# print ('second element', c[0][1].shape)
# print ('ingredients', c[0][2].shape)
# print ('steps', c[1].shape)
# print ('steps', c[1].dtype)
# # print ('Masks', c[2])

# Network

In [0]:
#Network defining

class RecipeNet(nn.Module):
    def __init__(self, vocab_size, feature_size, sequence_length):
        super(RecipeNet, self).__init__()
        self.vocab_size = vocab_size      #Size of OpenAI keys, see whether it can be changed to some smaller number, because it is too big
        self.feature_size = feature_size
        self.sequence_length=sequence_length
        
        #Encoder layers
        self.encoder_name = nn.Embedding(self.vocab_size, self.feature_size)
        self.encoder_calorie = nn.Embedding(10, self.feature_size)
        self.encoder_ingr = nn.Embedding(self.vocab_size, self.feature_size)
        
        #Batch Norm
        self.batch_norm = nn.BatchNorm2d(self.sequence_length)   #originally 41

        #GRU layers, for new structure 15
        self.gru_cat = nn.GRU(self.feature_size, self.vocab_size//10, batch_first=True)

        #Sequential Linear Layers, Try adding a ReLU somehow (would provide good results)
        #Also try adding BatchNorm1d
        self.linear_layer = nn.Sequential(nn.Linear(self.vocab_size//10, self.vocab_size//5),
                                          nn.ReLU(), nn.Linear(self.vocab_size//5, self.vocab_size))                                        
        
        self.best_accuracy = -1
    
    def forward(self, x, hidden):
        #x will be a concatenated set [batch_size, sequence_length, feature_size]

        x, hidden = self.gru_cat(x, hidden)

        return x, hidden

    def forward_emb(self, x_name, x_calorie, x_ingr):
        #x is a tuple and returns (name, calorie, ingredients) same tuple with embeddings
        #[N, sequence_length]  #[N, 20, sequence_length]   #[N, 20, 18, sequence_length]

        #Maybe later replace mean with something else
        x_ingr_shape=x_ingr.shape
        x_ingr = torch.sum(x_ingr, dim=2, keepdim=False)//x_ingr_shape[2]
        # x_ingr=x_ingr.reshape(x_ingr_shape[0], x_ingr_shape[1]*x_ingr_shape[2], x_ingr_shape[3])  #[N 20x18 sequence_length]

        #Taking encoder values
        x_name = self.encoder_name(x_name)
        x_calorie = self.encoder_calorie(x_calorie)
        x_ingr = self.encoder_ingr(x_ingr)

        #Processing shapes and returning concatenated value
        x_calorie = x_calorie.unsqueeze(1)

        #Concatenating name, calorie and ingr
        x = torch.cat((x_name, x_calorie, x_ingr), 1)   #[N (20+1+20) sequence_length feature_size]

        #Apply batch norm
        x = x.permute(0,2,1,3)
        x = self.batch_norm(x)
        x=x.permute(0,2,1,3)

        return x

    def forward_decoder(self, x):
        #x will be of shape [N, sequence_length, max_steps_len]
        #Return softmax probabilities

        # x, hidden = self.gru_decoder(x, hidden)
        x=x.view(-1, self.vocab_size//10)
        x = self.linear_layer(x)

        # x=F.softmax(x, dim=1)
        x=torch.sigmoid(x)    #Calculates independent so while forming sentences we can set a threshold

        return x

    # Predefined loss function
    def loss(self, prediction, label, reduction='mean'):
        #Sizes have been taken care of while inputing,
        loss_val = F.binary_cross_entropy_with_logits(prediction, label, reduction=reduction)
        return loss_val

    # Saves the current model
    def save_model(self, file_path, num_to_keep=1):
        pt_util.save(self, file_path, num_to_keep)

    # Saves the best model so far
    def save_best_model(self, accuracy, file_path, num_to_keep=1):
        if accuracy > self.best_accuracy:
            self.save_model(file_path, num_to_keep)
            self.best_accuracy = accuracy

    def load_model(self, file_path):
        pt_util.restore(self, file_path)

    def load_last_model(self, dir_path):
        return pt_util.restore_latest(self, dir_path)

# Training and Testing Funcs

In [0]:
#Train and test functions:
import tqdm
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

def train(model, device, optimizer, train_loader, lr, epoch, log_interval=20):
    model.train()
    losses = []
    hidden = None
    hidden_decoder=None
    for batch_idx, (data, label) in enumerate(tqdm.tqdm(train_loader)):
        # Separates the hidden state across batches. 
        # Otherwise the backward would try to go all the way to the beginning every time.
        #Data unpacking and loading to device
        name, calorie, ingr = data
        name=name.to(device)
        calorie=calorie.to(device)
        ingr=ingr.to(device)
        label = label.to(device)      #Label shape [N, 40483 (vocab_size)]
        
        #Get the embedded versions of name, calorie and ingredients
        x = model.forward_emb(name, calorie, ingr)
        if hidden is not None:
            hidden = repackage_hidden(hidden)
        optimizer.zero_grad()
        #For loop over dim=1
        # for i in range(x.shape[1]):
        #     output, hidden = model.forward(x[:,i,:,:], hidden)    #Final out shape = [N, sequence, vocab_size//10]
        #x shape    [N (20+1+20x18) 1 feature_size]
        output, hidden=model.forward(x.squeeze(), hidden)   #Squeeze to remove sequence_length
        output= model.forward_decoder(hidden)
        #output shape   [N (20+1+20x18) vocab_size//10]
        #Pass through decoder... Think it through if hidden_decoder is required or not
        # print ('hidden shape', hidden.shape)
        # output= model.forward_decoder(output)    #output shape = [N, vocab_size]
        #Calculating loss
        loss = model.loss(output, label)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return np.mean(losses)

def test(model, device, test_loader, train_loader, vocab_size):
    print ('Entered test')
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        hidden = None
        hidden_decoder=None
        for batch_idx, (data, label) in enumerate(test_loader):
            name, calorie, ingr=data
            #Uploading to device
            name=name.to(device)
            calorie=calorie.to(device)
            ingr=ingr.to(device)
            data=(name, calorie, ingr)
            label = label.to(device)      #Label shape [N, 40483 (vocab_size)]
            #Get the embedded versions of name, calorie and ingredients
            x = model.forward_emb(name, calorie, ingr) 
            # For loop over dim=1
            # print ('Testing {}'.format(batch_idx))
            # for i in range(x.shape[1]):
            #     output, hidden = model.forward(x[:,i,:,:], hidden)    #out_shape=[N, seq, vocab_size//10]
            #Passing through decoder
            output, hidden=model.forward(x.squeeze(), hidden)
            output = model.forward_decoder(hidden)
            # print ('hidden shape', hidden.shape)
            # output = model.forward_decoder(output)    #[N, vocab_size]
            #Calculating loss
            test_loss += model.loss(output, label).item()
            #Below lines of evaluation need to be changed
            pred = output
            #See what can be done here (To check for correct responses)
            correct_mask = pred.eq(label)
            # Change the definition of num_correct (run a for loop and keep a tolerance)
            num_correct = correct_mask.sum().item()
            correct += num_correct

    test_loss /= len(test_loader)
    test_accuracy = 100. * correct / (len(test_loader.dataset) * vocab_size)

    # print (len(test_loader.dataset), test_loader.dataset.sequence_length)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset) * vocab_size,
        100. * correct / (len(test_loader.dataset) * vocab_size)))
    return test_loss, test_accuracy

# Main.py

In [0]:
#Variables that will be used later on...

SEQUENCE_LENGTH = 1   #Keep sequence_length 1, else error
BATCH_SIZE = 200    #originally 512 
FEATURE_SIZE = 1000  # originally 1800, 1000
TEST_BATCH_SIZE = 100   #originally 100, see which is the fastest
EPOCHS = 0
LEARNING_RATE = 0.1   #originally 0.0025
WEIGHT_DECAY = 0.0005    #originally 0.0005, 0.00003
MOMENTUM=0.9
USE_CUDA = True
PRINT_INTERVAL = 20
e=2.718
LOG_PATH = BASE_PATH + 'logs/log_new_structure.pkl'
vocab_size=40483

In [0]:
#Main.py

def main():
    # SEQUENCE_LENGTH = 1   #Keep sequence_length 1, else error
    # BATCH_SIZE = 512    #originally 512 
    # FEATURE_SIZE = 1000  # originally 1800
    # TEST_BATCH_SIZE = 500   #originally 100, see which is the fastest
    # EPOCHS = 50
    # LEARNING_RATE = 0.1   #originally 0.0025
    # WEIGHT_DECAY = 0.0005    #originally 0.0005, 0.00003
    # MOMENTUM=0.9
    # USE_CUDA = True
    # PRINT_INTERVAL = 50
    # e=2.718
    # LOG_PATH = BASE_PATH + 'logs/log_epoch50.pkl'

    print ('Vocab size', vocab_size)
    data_train = Datasetloader(DATA_PATH + 'pp_recipes_chars_train.pkl', SEQUENCE_LENGTH, BATCH_SIZE, vocab_size)
    data_test = Datasetloader(DATA_PATH + 'pp_recipes_chars_test.pkl', SEQUENCE_LENGTH, TEST_BATCH_SIZE, vocab_size)

    use_cuda = USE_CUDA and torch.cuda.is_available()
    print ('torch cuda availability', torch.cuda.is_available())
    torch.cuda.empty_cache()

    device = torch.device("cuda" if use_cuda else "cpu")
    print('Using device', device)
    import multiprocessing
    num_workers = multiprocessing.cpu_count()
    print('num workers:', num_workers)

    kwargs = {'num_workers': num_workers,
              'pin_memory': True} if use_cuda else {}
    
    print('train, test loader variable declaration')
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=BATCH_SIZE,
                                               shuffle=False, **kwargs)
    test_loader = torch.utils.data.DataLoader(data_test, batch_size=TEST_BATCH_SIZE,
                                              shuffle=False, **kwargs)

    print ('model declaration')
    model = RecipeNet(vocab_size, FEATURE_SIZE, SEQUENCE_LENGTH).to(device)

    # Adam is an optimizer like SGD but a bit fancier. It tends to work faster and better than SGD.
    # We will talk more about different optimization methods in class.
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    # optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    # start_epoch = model.load_last_model(DATA_PATH + 'checkpoints_new_structure')
    # start_epoch = model.load_last_model(DATA_PATH + 'checkpoints')
    start_epoch=0

    print ('train_losses, test_losses and accuracy')
    try:
        train_losses, test_losses, test_accuracies, test_perplexities, train_perplexities = pt_util.read_log(LOG_PATH, ([], [], [], [], []))
        # train_losses, test_losses, test_accuracies, test_perplexities, train_perplexities = [], [], [], [], []
    except ValueError:
        train_losses, test_losses, test_accuracies, test_perplexities = pt_util.read_log(LOG_PATH, ([], [], [], []))
    print ('First test')
    test_loss, test_accuracy = test(model, device, test_loader, train_loader, vocab_size)
    print ('test-loss shape', test_loss)
    test_perplexity = e**test_loss
    test_losses.append((start_epoch, test_loss))
    test_accuracies.append((start_epoch, test_accuracy))
    test_perplexities.append((start_epoch, test_perplexity))
    train_perplexities=[]

    try:
        for epoch in range(start_epoch, EPOCHS + 1):
            print ('Epoch Number', epoch)
            lr = LEARNING_RATE * np.power(0.25, (int(epoch / 6)))
            train_loss = train(model, device, optimizer, train_loader, lr, epoch, PRINT_INTERVAL)
            test_loss, test_accuracy = test(model, device, test_loader, train_loader, vocab_size)
            test_perplexity = e**test_loss
            train_perplexity= e**train_loss
            train_losses.append((epoch, train_loss))
            test_losses.append((epoch, test_loss))
            test_accuracies.append((epoch, test_accuracy))
            train_perplexities.append((epoch, train_perplexity))
            test_perplexities.append((epoch, test_perplexity))
            pt_util.write_log(LOG_PATH, (train_losses, test_losses, test_accuracies, test_perplexities, train_perplexities))
            model.save_best_model(test_accuracy, BASE_PATH + 'checkpoints_lower_sized/%03d.pt' % epoch)

    except KeyboardInterrupt as ke:
        print('Interrupted')
    except:
        import traceback
        traceback.print_exc()
    finally:
        print('Saving final model')
        model.save_model(DATA_PATH + 'checkpoints_lower_sized/%03d.pt' % epoch, 0)
        
        #Add the plot functions, test it out and then run for a long epoch
        ep, val = zip(*train_losses)
        pt_util.plot(ep, val, 'Train loss', 'Epoch', 'Error')
        ep, val = zip(*test_losses)
        pt_util.plot(ep, val, 'Test loss', 'Epoch', 'Error')
        ep, val = zip(*test_accuracies)
        pt_util.plot(ep, val, 'Test accuracy', 'Epoch', 'Error')
        ep, val = zip(*test_perplexities)
        pt_util.plot(ep[1:], val[1:], 'Test Perplexity', 'Epoch', 'Perplexity')
        ep, val = zip(*train_perplexities)
        pt_util.plot(ep[1:], val[1:], 'Train Perplexity', 'Epoch', 'Perplexity')
        print ('Perplexity value', val)

        return model, device

# final_model, device = main()

# Recipe generation functions

In [0]:
#Technique list, place this somewhere on the top....
technique_list = ['rinse', 'bake',
    'barbecue',
    'blanch',
    'blend',
    'boil',
    'braise',
    'brine',
    'broil',
    'caramelize',
    'combine',
    'crock pot',
    'crush',
    'deglaze',
    'devein',
    'dice',
    'distill',
    'drain',
    'emulsify',
    'ferment',
    'freez',
    'fry',
    'grate',
    'griddle',
    'grill',
    'knead',
    'leaven',
    'marinate',
    'mash',
    'melt',
    'microwave',
    'parboil',
    'pickle',
    'poach',
    'pour',
    'pressure cook',
    'puree',
    'refrigerat',
    'roast',
    'saute',
    'scald',
    'scramble',
    'shred',
    'simmer',
    'skillet',
    'slow cook',
    'smoke',
    'smooth',
    'soak',
    'sous-vide',
    'steam',
    'stew',
    'strain',
    'tenderize',
    'thicken',
    'toast',
    'toss',
    'whip',
    'whisk',
]

In [0]:
import itertools
import difflib as diff

vocab_size=40483
vocab_size_local=vocab_size-5

#Indices
start_index=vocab_size_local+2
end_index=vocab_size_local+3
pad_index=vocab_size_local+1
start_step=vocab_size_local
end_step=vocab_size_local+4

tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
def launch_Decoder():
    Decoder={v:k for k,v in tokenizer.encoder.items()}
    vocab_size_local=len(tokenizer.encoder)
    # Decoder={end_step:'<s>', start_step:'</s>', pad_index: '</p>', start_index:'<R>', end_index:'</R>'}

    Decoder.update({
        end_step: '<s>',
        start_step: '</s>',
        pad_index: '</p>',
        start_index: '<R>',
        end_index: '</R>',
    })

    return Decoder


def Decode_return(x, Decoder):
    #x is integer
    temp_x=[Decoder[ii] for ii in x]

    return temp_x

#Initializing the tokenizer and the decoder
vocab_size_local=vocab_size-5

#Necessary functions for decoding recipe
#Decode string into required tokens
def string_tokenizer(name_str, calorie_str, ingr_str):
    #name is a string
    #calorie is a number
    #ingr_str is a list of ingredients
    # tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
    #Tokenize name
    name=tokenizer.tokenize(name_str)
    name=tokenizer.convert_tokens_to_ids(name)
    name=[start_index]+name+[end_index]

    #Tokenize ingredients
    #Contains several items, consider one at a time
    ingr=[]
    for count, ii in enumerate(ingr_str):
        ingr.append(tokenizer.tokenize(ii))
    #Convert tokens to ids
    for count, ii in enumerate(ingr):
        ingr[count]=tokenizer.convert_tokens_to_ids(ii)

    #Calorie, a number will be converted to either 0,1 or 2
    if calorie_str=="HIGH":
        calorie=2
    elif calorie_str=="MEDIUM":
        calorie=1
    elif calorie_str=="LOW":
        calorie=0

    return name, calorie, ingr, Decoder

#Make a function to return only tokens above a particular threshold
def top_probs_threshold_k(output, threshold):
    output_shape=output.shape
    output_list=[ii.item() if ii.item()>threshold else 0 for ii in output[0]]
    # print ('threshold fn _ output shape', output_shape, len(output_list))

    return output_list

#Function to return only the top max_steps_len probabilites
def top_p_probs(output, gold, threshold, ingredients, Decoder, n=max_steps_len):     #Not here output is a tensor, return a list
    top_p_probs=[]
    # output_shape=output.shape
    # print ('output_shape inside probs', output.shape)
    # output_list=output.squeeze().tolist()
    output_list=output
    # print ('output_list_shape', len(output_list))     #List of length [vocab_size], containing probs only
    indices=[k for k in range(len(output))]       #Indices
    #Considering probs in gold_standard
    x=[]
    for ii in range(len(gold)):
        count=0
        if output_list[gold[ii]]>=threshold:
            x.append(gold[ii])
            # print ('.'*10, 'Passed', '.'*10)
        else:
            if (Decoder[gold[ii]].replace('</w>','') in technique_list) or (Decoder[gold[ii]].replace('</w>','') in ingredients):
                # print ('.'*10, 'In here', '.'*10)
                x.append(gold[ii])
            else:
                try:  
                    for jj in range(gold[ii]-10,gold[ii]):
                        select=0
                        if (output_list[jj]>=threshold):
                            select=1
                            if count<=4:
                                x.append(gold[ii])
                            else:
                                x.append(jj)
                            # print ('.'*10, 'In here 2', '.'*10)
                            break
                        count+=1
                except IndexError:
                    for jj in range(gold[ii], gold[ii]+10):
                        select=0
                        if (output_list[jj]>=threshold):
                            select=1
                            if count<=4:
                                x.append(gold[ii])
                            else:
                                x.append(jj)
                            # print ('.'*10, 'In here 2', '.'*10)
                            break
                        count+=1
                count=0
                if select==0:
                    try:
                        for jj in range(gold[ii], gold[ii]+10):
                            if (output_list[jj]>=threshold):
                                select=1
                                if count<=4:
                                    x.append(gold[ii])
                                else:
                                    x.append(jj)
                                # print ('.'*10, 'In here 2', '.'*10)
                                break
                            count+=1
                    except IndexError:
                        pass
    # print ('len of x after choosing from gold', len(x))
    #Sorting along probability values and returning list of indices
    # x_ordered=[iterator for _,iterator in sorted(zip(output_list,indices), reverse=True)]
    # x+=x_ordered
    # x=x[:n]

    return x

def sentence_generator(output, gold):
    #output is a list of words outputed by the system
    #gold is the actual sentence
    #Return sequence with maximum score
    # existing_tech_list=[]
    # for ii in technique_list:
    #     if ii in gold:
    #         existing_tech_list.append(ii)

    # for jj in range(len(existing_tech_list)):
    #     existing_tech_list[jj]+='</w>'

    # for jj in range(len(ing)):
    #     ing[jj]+='</w>'
    special_chars=["<s>","</s>","<p>","</p>","<R>","</R>"]
    gold_temp=gold
    for ii in special_chars:
        gold_temp=gold_temp.replace(ii, '')
    # print ('gold sentence\n', gold_temp.replace('</w>', ' '))
    # print ('Generating permutations')
    outputs=itertools.permutations(output)    #All the permutations 
    # print ('outputs length', len(output))                      #Get the length of outputs
    #Set an upper limit on the check,say 10000
    count=0
    max_iterations=10000

    # chars=['<R>','</R>','<s>','</s>']
    # for ii in chars:
    #     gold=gold.replace(ii, '')
    gold_output=""
    # print ('Gold just before the loop\n', gold)
    similarity_score=0
    for ii in outputs:
        #ii is a tuple of words 
        temp_sentence=''.join(ii)     #Form a sentence

        score_temp=diff.SequenceMatcher(None, gold, temp_sentence).ratio()
        if score_temp>similarity_score:
            similarity_score=score_temp
            gold_output=temp_sentence

        if similarity_score>0.8:
            break

        if count>max_iterations:
            break
        
        count+=1
    # print ('gold_output', gold_output)
    return gold_output, similarity_score, gold_temp.replace('</w>',' ')     #Best generated sentence

#Need to add a technique list
def generate_recipe(output, gold, Decoder, ingredients, threshold=0.9):
    # #Printing gold value probs
    # for ii in range(len(gold)):
    #     print ('Gold value prob', gold[ii], ':',output[gold[ii]], '\n')
    #     if (output[gold[ii]]<threshold):
    #         for jj in range(10):
    #             try:
    #                 print ('Nearby values', gold[ii]+jj-5, ':', output[gold[ii]+jj-5], '\n')
    #             except IndexError:
    #                 pass
    
    #gold is recipe token representation (Not the sentence)
    # output=top_probs_threshold_k(output, threshold)
    x=top_p_probs(output, gold, threshold, ingredients, Decoder, len(gold))      #To match len replace max_steps_len with len(gold)
    # print ('3')
    # print ('x_shape', len(x))
    # print ('x\n', x)
    # print ('gold\n', gold)

    #Generating gold sentence
    # tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
    string_x=Decode_return(x, Decoder)
    string_gold=Decode_return(gold, Decoder)
    sentence_gold=''.join(string_gold)
    # print ('Done converting')

    #Getting the best sentence
    print ('\n\n', '.'*40, 'Generating your sentence', '.'*40)
    sentence, score, gold_recipe=sentence_generator(string_x, sentence_gold)

    # print ('Sentence prepared')
    sentence=sentence.replace("</w>", " ")
    special_chars=["<s>","</s>","<p>","</p>","<R>","</R>"]
    for jj in special_chars:
        sentence=sentence.replace(jj,'')
    return sentence, score, gold_recipe

#Testing sequence generator
# x=generate_recipe(output, threshold)
# print ('x', x)

# print (tokenizer)
# print (Decoder[40480])
# print (Decoder[40482])

In [0]:
def prediction_generator(name, calorie, ingredients):
    #Load the device
    # print ('.'*10, 'Inside prediction generator','.'*10)
    torch.cuda.empty_cache()
    use_cuda = USE_CUDA and torch.cuda.is_available()
    # print ('torch cuda availability', torch.cuda.is_available())
    device = torch.device("cuda" if use_cuda else "cpu")
    # print ('Device', device)

    #Model on forward function mode only
    # print ('model declaration')
    model = RecipeNet(vocab_size, FEATURE_SIZE, SEQUENCE_LENGTH).to(device)
    model.load_model(BASE_PATH + 'checkpoints_epoch50/048.pt')
    # print ('2')
    model.eval()

    #Load the inputs to device
    # print ('3')
    x_name=name.to(device)      #[20]
    x_calorie=calorie.to(device)    #[1]
    x_ingr=ingredients.to(device)   #[20, 18]

    hidden=None

    #Shaping the inputs for the network
    #Shape modifications
    x_name=x_name.unsqueeze(0)
    x_name=x_name.unsqueeze(2)
    x_calorie=x_calorie.unsqueeze(0)
    x_calorie=x_calorie.unsqueeze(1)
    x_ingr=x_ingr.unsqueeze(0)
    x_ingr=x_ingr.unsqueeze(-1)

    # print ('x_name, x_calorie, x_ingr shape', x_name.shape, x_calorie.shape, x_ingr.shape)

    #Embedding
    x = model.forward_emb(x_name, x_calorie, x_ingr)
    # print ('Shape of x', x.shape)
    #For loop over dim=1
    for i in range(x.shape[1]):
        output, hidden = model.forward(x[:,i,:,:], hidden)    #Final out shape = [1, sequence, vocab_size//10]
    # print ('x shape', x.shape)
    # output, hidden=model.forward(x.squeeze(2), hidden)
    # print ('output and hidden shapes after model.forward', output.shape, hidden.shape)
    output=model.forward_decoder(output)    #output shape = [1, vocab_size]

    # print ('.'*10, 'Completed prediction generator','.'*10)
    return output


In [0]:
#Returning name,calorie_level and ingredient given integer input
#For Demo purposes

def InputsGenerator(x):
    if x==0:
        name_in="aromatic basmati rice rice cooker"
        calorie_in="LOW"
        ingredients_in=['basmati rice', 'water', 'salt', 'cinnamon stick', 'green cardamom pods']
    elif x==1:
        name_in="arriba   baked winter squash mexican style"
        calorie_in="MEDIUM"
        ingredients_in=['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']
    elif x==2:
        name_in="apple a day  milk shake"
        calorie_in="HIGH"
        ingredients_in=['milk', 'vanilla ice cream', 'frozen apple juice concentrate', 'apple']
    elif x==3:
        name_in="Pomberrytini"
        calorie_in="LOW"
        ingredients_in=['pomegranate-blueberry juice', 'cranberry juice', 'vodka', 'raspberry schnapps', 'ice', 'orange slice']
    elif x==4:
        name_in="how i got my family to eat spinach  spinach casserole"
        calorie_in="MEDIUM"
        ingredients_in=['frozen chopped spinach', 'egg', 'salt', 'black pepper', 'onion', 'sharp cheddar cheese', 'condensed cream of mushroom soup', 'crouton']
    elif x==5:
        name_in="chicken lickin  good  pork chops"
        calorie_in="HIGH"
        ingredients_in=['lean pork chops', 'flour', 'salt', 'dry mustard', 'garlic powder', 'oil', 'chicken rice soup']
    elif x==6:
        name_in="boat house  collard greens"
        calorie_in="MEDIUM"
        ingredients_in=['collard greens', 'brown sugar', 'molasses', 'hot sauce', 'whiskey', 'ham hock', 'salt']
    elif x==7:
        name_in="better then bush s  baked beans"
        calorie_in="MEDIUM"
        ingredients_in=['great northern bean', 'chicken bouillon cubes', 'dark brown sugar', 'molasses', 'cornstarch', 'onion', 'garlic powder', 'mustard powder', 'chili powder', 'salt', 'black pepper', 'bacon', 'water']
    elif x==8:
        name_in="beat this  banana bread"
        calorie_in="HIGH"
        ingredients_in=['sugar','unsalted butter','bananas','eggs','fresh lemon juice','orange rind','cake flour','baking soda','salt']
    elif x==9:
        name_in="get the sensation  brownies"
        calorie_in="HIGH"
        ingredients_in=['butter','sugar','vanilla','eggs','all-purpose flour','baking cocoa','baking powder','salt','miniature peppermint patties']
    elif x==10:
        name_in="keep it going  german friendship cake"
        calorie_in="HIGH"
        ingredients_in=['flour','water','dry yeast','milk','sugar','eggs','vegetable oil','baking soda','baking powder','salt','cinnamon','vanilla','crushed pineapple','raisins','nuts','butter','brown sugar']
    elif x==11:
        name_in="killer  lasagna"
        calorie_in="HIGH"
        ingredients_in=['italian sausage','ground beef','garlic','dried basil','salt','whole tomato','tomato paste','ricotta cheese','cottage cheese','parmesan cheese','parsley flakes','eggs','ground black pepper','lasagna noodle','mozzarella cheese']
    elif x==12:
        name_in="make that chicken dance  salsa pasta"
        calorie_in="HIGH"
        ingredients_in=['tomatoes','garlic','onion','button mushrooms','hot sauce','dried oregano','dried basil','fresh parsley','water','salsa','chicken breasts','olive oil','fresh ground black pepper','parmesan cheese','pasta']
    elif x==13:
        name_in="real  italian bolognese sauce"
        calorie_in="HIGH"
        ingredients_in=['bacon','onion','celery','carrot','garlic','butter','olive oil','lean ground beef','ground pork','beef consomme','dry white wine','crushed tomatoes','salt','black pepper','rubbed sage','oregano','red pepper flakes','nutmeg','milk','penne pasta']
    elif x==14:
        name_in="the elvis  smoothie"
        calorie_in="MEDIUM"
        ingredients_in=['banana','natural-style peanut butter','2% low-fat milk','sugar-free non-fat vanilla yogurt']
    elif x==15:
        name_in="truth serum  margaritas"
        calorie_in="MEDIUM"
        ingredients_in=['limeade','lime juice','tequila','triple sec','lime wedge','grand marnier']
    else:
        raise Exception("Enter a correct number")

    return name_in, calorie_in, ingredients_in

# Recipe Generation

In [0]:
#Feeding the output value into the generate sequence and see if it makes any sense
# #Constant Declaration, Note the below values must match with that of main.py

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

#Input the name, ingredients and calories

#########################################################################
# # #Case:1, (tough)
# name_in="aromatic basmati rice rice cooker"     #Enter name string
# calorie_in="LOW"    #Enter value for calorie
# ingredients_in= ['basmati rice', 'water', 'salt', 'cinnamon stick', 'green cardamom pods']   #Enter list of ingredients (Must be a list)
# ingredients_in= ['basmati rice', 'water', 'salt', 'apple', 'bananas']

# # Case:2 (tough)
# name_in="arriba   baked winter squash mexican style"
# calorie_in="MEDIUM"
# # ingredients_in=['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']
# ingredients_in=['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'pepper', 'apple']

#Case:3 (easy)
# name_in="apple a day  milk shake"
# calorie_in="HIGH"
# ingredients_in=['combine ingredients in blender', 'cover and blend until smooth', 'sprinkle with ground cinnamon', 'makes about 2 cups']    #Complete list of ingredients
# ingredients_in=['combine ingredients in blender', 'cover and blend until smooth', 'sprinkle pepper', 'makes about 3 cups']
# ingredients_in=['combine ingredients in blender', 'cover and blend until smooth']

#Case:4 Reference paper example
# name_in="Pomberrytini"
# calorie_in="LOW"
# ingredients_in=['pomegranate-blueberry juice', 'cranberry juice']   #This list is incomplete

# # #Finding best case example
# name_in="truth serum  margaritas"
# calorie_in="MEDIUM"
# ingredients_in=['butterscotch chips', 'chinese noodles', 'salted peanuts']
###################################################################################
recipe_number=10

name_in, calorie_in, ingredients_in=InputsGenerator(recipe_number)

ingredients_in_copy=ingredients_in.copy()

Decoder=launch_Decoder()

name, calorie, ingredients, Decoder = string_tokenizer(name_in, calorie_in, ingredients_in)

#Preprocess input tensors to maximum lengths
remainder=max_name_len-len(name)
name=name+[0]*remainder

ingr=[]
for ii in ingredients:
    while len(ii)<max_ingr_inner_len:
        ii.append(0)
    ingr.append(ii)

while len(ingr)<max_ingr_len:
    ingr.append([0]*max_ingr_inner_len)

ingredients=ingr

#Find gold_standard in training and testing both
found_data=0

    #Search for recipe steps in traning
data_file=DATA_PATH+'pp_recipes_chars_train.pkl'
with open(data_file, 'rb') as data_pkl:
    dataset = pickle.load(data_pkl)

name_tokens=dataset['name_tokens']
step_tokens=dataset['step_tokens']

assert len(name_tokens)==len(step_tokens)   #len should be same for both

for ii in range(len(name_tokens)):
    if name_tokens[ii]==name:
        found_data=1
        break
gold_steps=step_tokens[ii]

    #If recipe not in training dataset, search in testing dataset
if found_data==0:
    data_file=DATA_PATH+'pp_recipes_chars_test.pkl'
    with open(data_file, 'rb') as data_pkl:
        dataset=pickle.load(data_pkl)
    
    name_tokens=dataset['name_tokens']
    step_tokens=dataset['step_tokens']

    assert len(name_tokens)==len(step_tokens)
    # print ('Length of name tokens', len(name_tokens))

    for ii in range(len(name_tokens)):
        if name_tokens[ii]==name:
            found_data=1
            break
    gold_steps=step_tokens[ii]

if found_data==0:
    #Generate an error
    raise Exception('Correct the name')

# print ('Removing zeros from gold_steps')
#Remove all the zeros from gold_steps
gold_steps=[val for val in gold_steps if val!=0]

# print ('gold steps\n', gold_steps)
# print ('gold steps length', len(gold_steps))

# print ('name, calorie, ingredient', name, '\n', calorie, '\n', ingredients)
#Convert the lists to torch tensors
name=torch.tensor(name, dtype=torch.long)
calorie=torch.tensor(calorie, dtype=torch.long)
ingredients=torch.tensor(ingredients, dtype=torch.long)

# print ('input shapes', name.shape, calorie.shape, ingredients.shape)

# print ('Vocab size', vocab_size)

output=prediction_generator(name, calorie, ingredients)
# print ('Shape of output outside prediction generator\n', output.shape)

pos_values=0
for ii in range(output.shape[1]):
    if output[0, ii]>0.8:
        pos_values+=1

# print ('pos_values', pos_values)
#Sequence_decoding
output=output.squeeze().tolist()
threshold=0.7
# print ('ingredients_in', ingredients_in_copy)
recipe, score, actual_recipe=generate_recipe(output, gold_steps, Decoder, ingredients_in_copy, threshold)

#Printing the recipe
print ("\n\n")
print ("Name of the dish:\t", name_in)
print ("\nCalorie:\t\t", calorie_in)
print ("\nList of ingredients:\t", ingredients_in_copy)
print ('\nSequence_Match score:\t', score)
print ('\nDecoder recipe:\t\t', recipe)
print ("\nActual recipe:\t\t", actual_recipe)

Restoring:
encoder_name.weight -> 	torch.Size([40483, 1000]) = 161MB
encoder_calorie.weight -> 	torch.Size([10, 1000]) = 0MB
encoder_ingr.weight -> 	torch.Size([40483, 1000]) = 161MB
batch_norm.weight -> 	torch.Size([1]) = 0MB
batch_norm.bias -> 	torch.Size([1]) = 0MB
batch_norm.running_mean -> 	torch.Size([1]) = 0MB
batch_norm.running_var -> 	torch.Size([1]) = 0MB
batch_norm.num_batches_tracked -> 	torch.Size([]) = 0MB
gru_cat.weight_ih_l0 -> 	torch.Size([12144, 1000]) = 48MB
gru_cat.weight_hh_l0 -> 	torch.Size([12144, 4048]) = 196MB
gru_cat.bias_ih_l0 -> 	torch.Size([12144]) = 0MB
gru_cat.bias_hh_l0 -> 	torch.Size([12144]) = 0MB
linear_layer.0.weight -> 	torch.Size([8096, 4048]) = 131MB
linear_layer.0.bias -> 	torch.Size([8096]) = 0MB
linear_layer.2.weight -> 	torch.Size([40483, 8096]) = 1311MB
linear_layer.2.bias -> 	torch.Size([40483]) = 0MB

Restored all variables
No new variables
Restored /gdrive/My Drive/colab_files/dl_project/checkpoints_epoch50/048.pt


 ......................

# Training on smaller dataset

In [0]:
#Training smaller dataset smaller dataset

EPOCHS = 1000
LEARNING_RATE = 0.01   #originally 0.0025
WEIGHT_DECAY = 0.0005    #originally 0.0005
MOMENTUM=0.9
USE_CUDA = True

#Device setup
use_cuda = USE_CUDA and torch.cuda.is_available()
print ('torch cuda availability', torch.cuda.is_available())
torch.cuda.empty_cache()

device = torch.device("cuda" if use_cuda else "cpu")

calorie=[1,4,6,2,7]
name=[2,3,4,5,6]
ingredients=[[11,12,13],[14,15,16],[17,18,19],[18,17,13],[12,13,19]]    #One set of ingredients per recipe
steps = [[20,21,22,25,26],[30,31,41,51],[25,27,29,33],[27,29,20],[30,32,33,45,56]]

model=RecipeNet(100,100,1).to(device)    #vocab_size, feature_size, sequence_length
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# optimizer =optim.SGD(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)
#Prepare the training tensor
combined_train_data_labels=[]
for i in range(5):
    train_data=[[name[i]]+[0]*19, calorie[i], ingredients[i]+15*[0]]
    labels=[0]*100
    for j in range(len(steps[i])):
        labels[steps[i][j]]=1
    combined_train_data_labels.append((train_data, labels))
print (len(combined_train_data_labels))

model.train()
for epoch in range(EPOCHS):
    losses=[]
    hidden=None
    for batch_idx, (data, label) in enumerate(combined_train_data_labels):
        # lr = LEARNING_RATE * np.power(0.7, (int(epoch / 20)))
        lr=LEARNING_RATE
        x_name=torch.tensor(data[0], device=device)   #[20]
        x_calorie=torch.tensor(data[1], device=device)    #[1]
        x_ingr=torch.tensor(data[2], device=device)       #[18]
        label=torch.tensor(label, dtype=torch.float32, device=device)        #[100]

        #Shape modifications
        x_name=x_name.unsqueeze(0)
        x_name=x_name.unsqueeze(2)
        x_calorie=x_calorie.unsqueeze(0)
        x_calorie=x_calorie.unsqueeze(1)
        x_ingr=x_ingr.unsqueeze(0)
        x_ingr=x_ingr.unsqueeze(1)
        x_ingr=x_ingr.unsqueeze(3)
        label=label.unsqueeze(0)

        #Embedding
        x = model.forward_emb(x_name, x_calorie, x_ingr)
        optimizer.zero_grad()
        #For loop over dim=1
        for i in range(x.shape[1]):
            output, hidden = model.forward(x[:,i,:,:], hidden)    #Final out shape = [N, sequence, vocab_size//10]
        #Pass through decoder... Think it through if hidden_decoder is required or not
        output, hidden=model.forward(x.squeeze(2), hidden)
        output=model.forward_decoder(hidden)    #output shape = [N, vocab_size]
        #Calculating loss
        loss = model.loss(output, label, reduction='sum')
        # a=list(model.parameters())[0].clone()
        losses.append(loss.item())
        loss.backward(retain_graph=True)
        optimizer.step()
        # b=list(model.parameters())[0].clone()
        if batch_idx % 2==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx, 5,
              100. * batch_idx / 5, loss.item()))
    print ('Losses after one epoch', np.mean(losses))

#Print out the function that returns
